In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

Using TensorFlow backend.


In [4]:
path = '../data/'
savepath = '../data/'
images = glob.glob(path+'sources/nparrs_384/*.npy')
totalImages = 1800

In [5]:
tile_sizes = np.loadtxt(path+'sources/complex_scenes/tile_sizes.txt', dtype='int')
images_sampled = {}
for tile in tile_sizes:
    if tile[2] > 150000:
        for i in range(30):
            images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])

In [11]:
def myGenerator(batch_size):
    while True:
        #index_list = random.sample(range(1, totalImages), batch_size)
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy' #images[i]
            frame = np.load(frame)
            #tile_index = np.random.randint(0, 199)
            tile_index = random.choice(images_sampled[i])
            #print(i, tile_index, frame.shape, images_sampled[i])
            alldata_x.append(tile_index*totalImages+i)
            alldata_y.append(frame[tile_index])
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        yield alldata_y, alldata_y
#x = myGenerator()
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [12]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(384, 384, 3), padding="valid"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, kernel_size=3, subsample=(2, 2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, kernel_size=3, subsample=(2, 2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, kernel_size=3, subsample=(2, 2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, kernel_size=3, subsample=(2, 2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(1, kernel_size=3, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))


        #Decoder
        #model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(ZeroPadding2D(padding=(2, 2)))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        #model.add(ZeroPadding2D(padding=(2, 2)))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        #model.add(ZeroPadding2D(padding=(3, 3)))
        model.add(UpSampling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, X_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
    def build_discriminator(self):
        img   = Input(shape=(192, 192, 3))
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(192, 192, 3)))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64,  (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (6, 6),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(32))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1))
        
        output    = model(img)
        
        model3 = Model(img, output)
        return model3

    def train(self, epochs, batch_size=128, sample_interval=50):
        random.seed(10)
        
        # Load the dataset
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            
            # Adversarial ground truths
            valid = np.ones((batch_size, 1))
            fake  = np.zeros((batch_size, 1))
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(X_train)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(X_train, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch(X_train, [X_train, valid])

            # Plot the progress
            print ("%d [D loss: %f, mean_acc: %.2f%% real_acc: %.2f%% fake_acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
                self.discriminator.save_weights(savepath+'weights/discriminator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, Y_train = next(myGenerator(8))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4]])
        combined = np.hstack(combined.reshape(5, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([Y_train[0], Y_train[1], Y_train[2], Y_train[3], Y_train[4]])
        combined = np.hstack(combined.reshape(5, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)

In [ ]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(100000, 8, 100)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, strides=(2, 2), kernel_size=3)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, strides=(2, 2), kernel_size=3)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, strides=(2, 2), kernel_size=3)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, strides=(2, 2), kernel_size=3)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 382, 382, 64)      1792      
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 382, 382, 64)      0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 382, 382, 64)      256       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 190, 190, 128)     73856     
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 190, 190, 128)     0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 190, 190, 128)     512       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 94, 94, 128)       147584    
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:198: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:202: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch  1  G loss  0.7618593
Epoch  2  G loss  0.29116008
Epoch  3  G loss  0.35995567
Epoch  4  G loss  0.27188468
Epoch  5  G loss  0.26688558
Epoch  6  G loss  0.21957155
Epoch  7  G loss  0.18818226
Epoch  8  G loss  0.20432904
Epoch  9  G loss  0.19924057
Epoch  10  G loss  0.17930081
Epoch  11  G loss  0.2912131
Epoch  12  G loss  0.21690708
Epoch  13  G loss  0.18871263
Epoch  14  G loss  0.2843777
Epoch  15  G loss  0.15683669
Epoch  16  G loss  0.19552423
Epoch  17  G loss  0.16363561
Epoch  18  G loss  0.14530428
Epoch  19  G loss  0.52909213
Epoch  20  G loss  0.28074104
Epoch  21  G loss  0.14673892
Epoch  22  G loss  0.15906462
Epoch  23  G loss  0.15422048
Epoch  24  G loss  0.22080258
Epoch  25  G loss  0.1800712
Epoch  26  G loss  0.18767953
Epoch  27  G loss  0.1330104
Epoch  28  G loss  0.15831779
Epoch  29  G loss  0.17756009
Epoch  30  G loss  0.12947863
Epoch  31  G loss  0.19395897
Epoch  32  G loss  0.13084634
Epoch  33  G loss  0.19568107
Epoch  34  G loss  0.147

In [ ]:
import time
for i in range(100):
    x = myGenerator(1)
    xtest, ytest = next(x)
    #print('xtrain shape:',xtest.shape)
    #print('ytrain shape:',ytest.shape)
    pred = cgan.generator.predict(xtest)
    pred = pred*127.5 + 127.5
    pred = pred.astype(int)
    print(pred.dtype)
    plt.imshow(pred[0])
    plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    plt.imshow(ytest[0])
    plt.show()
    #break
    #time.sleep(1)